In [1]:
# basic
import pandas as pd
import numpy as np

# ploting
import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#import plotly.graph_objs as go
#from mpl_toolkits.mplot3d import Axes3D 

# models
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct
from sklearn.linear_model import BayesianRidge, ElasticNet, Lasso
from sklearn.neighbors import KNeighborsRegressor
import sklearn.gaussian_process as gp
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor

# helpers etc.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, max_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from itertools import combinations
import time

# display
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

#from statsmodels.nonparametric.kernel_regression import KernelReg
#%matplotlib notebook

C:\Users\sevcn\AppData\Local\Temp\ipykernel_18496\435150283.py:31: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Helper functions

In [2]:
def powerset(l):
    S = []
    for size in range(0,len(l)+1):
        S.extend(list(combinations(l, size)))
    return S

In [3]:
def df_to_features_and_labels(df, only_positive_features=True, scale_features=True):
    r"""Separate df into features (X) and labels (Y). Convert weather to 0/1. Convert time to two continous variables ( https://stats.stackexchange.com/questions/245866/is-hour-of-day-a-categorical-variable )"""
    Y = df.iloc[:, 4]
    X = df.drop(columns=['date', df.columns[4]])
    X['weather'] = [0 if x == 'bad' else '1' for x in X['weather']]
    X['weather'] = X['weather'].astype(int)
    # https://stats.stackexchange.com/questions/245866/is-hour-of-day-a-categorical-variable
    X['time_x'] = np.sin(2*np.pi*X['time']/24)
    X['time_y'] = np.cos(2*np.pi*X['time']/24)
    X.drop(columns=['time'], inplace=True)

    
    if only_positive_features and scale_features: # use minmax scaler that scales the data to 0,1 interval
        feature_scaler = MinMaxScaler()
        X[:] = feature_scaler.fit_transform(X)        
    elif only_positive_features:
        X['time_x'] += 1
        X['time_y'] += 1 
    elif scale_features: 
        feature_scaler = StandardScaler()
        X[:] = feature_scaler.fit_transform(X)
        #X_train = feature_scaler.fit_transform(X_train)
        #X_test = feature_scaler.transform(X_test)
        
    return X, Y

### Settings

In [4]:
f = open("data\\counters_per_route.txt", encoding="utf8")
route_counters = {}

for l in f:
    if l.startswith("#") or (l == "\n"):
        continue
    ss = l.strip().split(";")
    route_id = ss[0] 
    #route_id = int(route_id)
    cs = ss[1:]  
    cs = list(map(lambda x: x.strip(), cs))
    if cs != ['']:
        route_counters[route_id] = cs
    
route_counters  

{'Dunajska (from centre)': ['1003-116-1', '9000000656-1', '9000000655-2'],
 'Dunajska (to centre)': ['1004-136-1', '9000000656-2', '9000000655-1'],
 'Ižanska (from centre)': ['1040-236-1', '9000000820-1', '9000001506-1'],
 'Ižanska (to centre)': ['1040-236-2', '9000000820-2', '9000001506-2'],
 'Slovenska (from centre)': ['1026-136-1', '9000000619-1'],
 'Slovenska (to centre)': ['1025-116-1', '9000000619-2'],
 'Škofije (towards Koper)': ['686-1', '9000001092-1'],
 'Škofije (towards Trieste)': ['686-2', '9000001092-2']}

In [5]:
#route_names = ['Dunajska1', 'Dunajska2', 'Ig1', 'Ig2',
#               'Koper1', 'Koper2', 'Slovenska1', 'Slovenska2']

route_names = list(route_counters.keys())
route_names

['Dunajska (from centre)',
 'Dunajska (to centre)',
 'Ižanska (from centre)',
 'Ižanska (to centre)',
 'Slovenska (from centre)',
 'Slovenska (to centre)',
 'Škofije (towards Koper)',
 'Škofije (towards Trieste)']

In [ ]:
#sns.set_theme()

In [6]:
only_positive_features = True
scale_features = False

In [7]:
# kernel ridge regression
candidate_kernels_kr = ['linear', 'rbf']
candidate_kernels_kr.append('chi2') if only_positive_features else None
parameters_kr = {'kernel': candidate_kernels_kr, 
                 "alpha": np.logspace(-5, 5, 11), 
                 "gamma": np.logspace(-5, 5, 11)}

# support vector regression
candidate_kernels_svr = ['rbf']
parameters_svr = {'kernel': candidate_kernels_svr, 
                  "C": [1e0, 1e1, 1e2, 1e3], 
                  "gamma": np.logspace(-5, 5, 11)}

# random forest regression
parameters_rfr = {"max_depth": np.arange(0,11)}


# Gaussian process regression
candidate_kernels_gpr = [RBF(l) for l in np.logspace(-1, 1, 4)]+ [DotProduct(sigma_0) for sigma_0 in np.logspace(-1, 1, 4)]
pararameters_gpr = {'kernel':candidate_kernels_gpr, 
                    "alpha": np.logspace(-5, 5, 11)}

# Bayesian ridge regression
parameters_brr = {'alpha_init':np.logspace(-5, 5, 11),
                  'lambda_init': np.logspace(-5, 5, 11)}

# k-nearest neighbors regression
parameters_knn = {'n_neighbors': list(range(1, 15)),
                  'weights': ['uniform', 'distance'],
                  'metric': ['euclidean', 'manhattan']}

# elastic-net regression
parameters_enr = {"alpha": np.logspace(-5, 5, 11),
                  "l1_ratio": np.arange(0.0, 1.0, 0.1)}

# Lasso regression
parameters_lr =  {"alpha": np.logspace(-5, 5, 11)}

# Ada-Boost regression: https://educationalresearchtechniques.com/2019/01/07/adaboost-regression-with-python/
#https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html#sphx-glr-auto-examples-ensemble-plot-adaboost-regression-py
parameters_ada = {'base_estimator': [KernelRidge(kernel="chi2"), KernelRidge(kernel="rbf"), None, KNeighborsRegressor()],
                  'n_estimators':[50,100,200],
                  'learning_rate':[.001,0.01,.1],
                  'random_state':[1]}

# bagging
# https://coderzcolumn.com/tutorials/machine-learning/scikit-learn-sklearn-ensemble-learning-bagging-and-random-forests
parameters_bag = {'base_estimator': [KernelRidge(kernel="chi2"), KernelRidge(kernel="rbf"), None, KNeighborsRegressor()], # zakaj KernelRidge ne dela s chi2 kernelom??
                  'n_estimators': [50,100,200],
                  #'max_samples': [0.5,1.0, n_samples//2,],
                  #'max_features': [0.5,1.0, n_features//2,],
                  #'bootstrap': [True, False],
                  #'bootstrap_features': [True, False]
                 }

# gradient boosting
parameters_gbr = {'n_estimators': [50,100,200],
                  'learning_rate':[.001,0.01,.1],
                  #'max_samples': [0.5,1.0, n_samples//2,],
                  #'max_features': [0.5,1.0, n_features//2,],
                  #'bootstrap': [True, False],
                  #'bootstrap_features': [True, False]
                 }



In [8]:
models = {'krr': KernelRidge(),
          'svr': SVR(),
          'rfr': RandomForestRegressor(n_estimators=100, oob_score=True),
          'gpr': GaussianProcessRegressor(),
          'bayes': BayesianRidge(),
          'knn': KNeighborsRegressor(),
          'elastic': ElasticNet(),
          'lasso': Lasso(),
          'ada':AdaBoostRegressor(),
          'bag':BaggingRegressor(),
          'gbr':GradientBoostingRegressor(),
         }


grids = {'krr': parameters_kr,
         'svr': parameters_svr,
         'rfr': parameters_rfr,
         'gpr': pararameters_gpr,
         'bayes': parameters_brr,
         'knn': parameters_knn,
         'elastic': parameters_enr,
         'lasso': parameters_lr,
         'ada': parameters_ada,
         'bag': parameters_bag,
         'gbr': parameters_gbr
        }

### Regression

In [9]:
basic_features = [[], ['workday', 'weather', 'time_x', 'time_y']]

df_results = pd.DataFrame(columns = ['route', 'label', 'features', 'model', 'MSE_train', 'MSE_test', 'max_error_train', 'max_error_test', 'R2_train', 'R2_test', 'best_params', 'best_score'])

for route in route_names:

    print("*********************")
    print("*********************")
    print('route ', route)
    print("*********************")
    print("*********************")
    
    file_name = r'data\route_' + route + '_counters.csv'
    df = pd.read_csv(file_name)
    #df = df.iloc[:,4:]
    df.dropna(inplace=True)

    X_full, Y_full = df_to_features_and_labels(df, only_positive_features, scale_features)  
    
    percent_test = 0.3
    # reproducible
    #     X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=percent_test, random_state=42)
    # random
    
    # the same train_test_split for all models - more fair comparison
    X_train, X_test, Y_train, Y_test = train_test_split(
                X_full, Y_full, test_size=percent_test)
    

    telraam_counters = df.columns[5:]
    ilc_counter = df.columns[4]
    telraam_powerset = powerset(telraam_counters)
    
    for c in telraam_powerset:
        for basic in basic_features:
            if not basic and not c:
                continue
        
            
            features = basic + list(c)
            X_train_f = X_train[features]
            X_test_f = X_test[features]
        
        
            for model in models:
                
                print("*********************")
                print("Predicting", ilc_counter, "with", features, "using", model)
                print()
                
                regmod = GridSearchCV(models[model], grids[model], scoring='r2')
                
                if model == 'rfr': # regression forests -  to avoid UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
                    regmod.fit(X_train_f.values, Y_train)
                    Y_train_pred = regmod.predict(X_train_f.values)
                    Y_test_pred = regmod.predict(X_test_f.values)
                    
                    print(model, "feature importance: ")
                    print({name: round(importance, 3)
                        for name, importance in zip(X_train_f.columns, regmod.best_estimator_.feature_importances_)})
                    
                else:
                    regmod.fit(X_train_f, Y_train)
                    Y_train_pred = regmod.predict(X_train_f)
                    Y_test_pred = regmod.predict(X_test_f)
                    
                    
                R2_train = r2_score(Y_train_pred, Y_train.values)
                R2_test = r2_score(Y_test_pred, Y_test.values)
                
                MSE_train = mean_squared_error(Y_train_pred, Y_train.values)
                MSE_test = mean_squared_error(Y_test_pred, Y_test.values)
                
                max_error_train = max_error(Y_train_pred, Y_train.values)
                max_error_test = max_error(Y_test_pred, Y_test.values)
                
                print(model, "train dataset R2:", R2_train)
                print(model, "test dataset R2:", R2_test)
                print(model,'best parameters: ', regmod.best_params_)
                print(model, 'best score: ', regmod.best_score_)
            
                print()
                
                d = {'route': route, 
                     'label': ilc_counter,
                     'features': features, 
                     'model': model, 
                     'MSE_train': MSE_train, 
                     'MSE_test': MSE_test, 
                     'max_error_train': max_error_train, 
                     'max_error_test': max_error_test,
                     'R2_train': R2_train,
                     'R2_test': R2_test, 
                     'best_params': regmod.best_params_,
                     'best_score': regmod.best_score_}
                
                df_results = df_results.append(d, ignore_index=True)

                


*********************
*********************
route  Dunajska (from centre)
*********************
*********************


FileNotFoundError: [Errno 2] No such file or directory: 'data\\route_Dunajska (from centre)_counters.csv'

In [ ]:
df_results.sort_values(by="R2_test", ascending=False)

,route,label,features,model,MSE_train,MSE_test,max_error_train,max_error_test,R2_train,R2_test,best_params,best_score
220,Ižanska (from centre),1040-236-1,"[workday, weather, time_x, time_y, 9000000820-...",krr,478.317502,823.313889,96.496660,118.525481,0.947349,0.898817,"{'alpha': 1e-05, 'gamma': 0.001, 'kernel': 'ch...",0.936297
429,Škofije (towards Trieste),686-2,"[workday, weather, time_x, time_y, 9000001092-2]",krr,662.940317,618.411635,156.826350,104.835103,0.884937,0.892481,"{'alpha': 0.0001, 'gamma': 0.01, 'kernel': 'ch...",0.873610
330,Slovenska (from centre),1026-136-1,"[workday, weather, time_x, time_y, 9000000619-1]",krr,426.227906,425.557462,114.765205,98.300902,0.897937,0.892127,"{'alpha': 0.001, 'gamma': 0.01, 'kernel': 'chi2'}",0.896049
439,Škofije (towards Trieste),686-2,"[workday, weather, time_x, time_y, 9000001092-2]",gbr,572.495366,615.591135,129.867713,92.303960,0.898961,0.889926,"{'learning_rate': 0.1, 'n_estimators': 100}",0.868458
431,Škofije (towards Trieste),686-2,"[workday, weather, time_x, time_y, 9000001092-2]",rfr,252.548502,630.568222,105.657412,111.500927,0.957053,0.887959,{'max_depth': 9},0.870514
...,...,...,...,...,...,...,...,...,...,...,...,...
39,Dunajska (from centre),1003-116-1,[9000000655-2],elastic,11376.146588,12645.234103,316.811337,300.405319,-2.216306,-2.625111,"{'alpha': 100.0, 'l1_ratio': 0.5}",0.230423
40,Dunajska (from centre),1003-116-1,[9000000655-2],lasso,11376.357363,12644.923274,316.601347,300.100460,-2.237815,-2.649196,{'alpha': 100.0},0.230421
43,Dunajska (from centre),1003-116-1,[9000000655-2],gbr,9580.326645,12190.584146,329.016909,373.661477,-1.771399,-2.833580,"{'learning_rate': 0.01, 'n_estimators': 200}",0.273264
34,Dunajska (from centre),1003-116-1,[9000000655-2],svr,10687.977945,11935.862369,350.212901,329.498125,-1.709741,-2.853756,"{'C': 1000.0, 'gamma': 1e-05, 'kernel': 'rbf'}",0.274874


In [ ]:
df_best = pd.DataFrame()

for label in df_results.label.unique():
    df2 = df_results[df_results['label']==label]
    df_best = df_best.append(df2[df2['R2_test'] == df2['R2_test'].max()], ignore_index=True)

In [ ]:
if scale_features:
    df_results.to_csv("regression_results_scaling.csv", index=False)
    df_best.to_csv("regression_results_best_scaling.csv", index=False)
else:
    df_results.to_csv("regression_results.csv", index=False)
    df_best.to_csv("regression_results_best.csv", index=False)